In [1]:
import cryptography.hazmat.primitives.asymmetric.rsa as rsa
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import serialization
import binascii
import math
import textwrap
from Crypto.Util.number import *

# modified form https://gist.github.com/kazkansouh/e4d710c6a6928187323fa164bdd70401

def local_setup(oracle_prfix="0011111111", choose_plaintext=b"flag{this_is_a_sample_flag_for_testing!}"):
    print('Using local loop back oracle for testing')
    priv_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=1024,
        backend=default_backend()
    )
    pub_key = priv_key.public_key()
    pn = pub_key.public_numbers()
    ciphertext = pow(int.from_bytes(choose_plaintext, "big"), pn.e, pn.n)

    print('  keysize: {}'.format(priv_key.key_size))
    print('  e: {}'.format(pn.e))
    print('  n: {}'.format(pn.n))
    print('  p: {}'.format(priv_key.private_numbers().p))
    print('  q: {}'.format(priv_key.private_numbers().q))
    print('  d: {}'.format(priv_key.private_numbers().d))

    print('  c: {}'.format(hex(ciphertext)))
    print()
    
    if oracle_prfix == "pkcs1.5":
        prefix_nbits = 16
        prefix_num = 0x0002
    else:
        prefix_nbits = len(oracle_prfix)
        prefix_num = int(oracle_prfix, 2)

    def oracle(ct):
        if type(ct) == int:
            c = ct
        elif type(ct) == bytes:
            c = int.from_bytes(ct, 'big')
        elif type(ct) == str:
            c = int(ct, 16)
        else:
            print("Kidding?")
            return None

        d = priv_key.private_numbers().d
        m = pow(c, d, pn.n)
        padding_pos = pn.n.bit_length() - prefix_nbits
        return (m >> padding_pos) == prefix_num

    def _oracle(ct):
        c = int.from_bytes(ct, 'big')

        try:
            priv_key.decrypt(
                ct,
                padding.PKCS1v15()
            )
            return True
        except ValueError:
            return False

    return ciphertext, oracle, pn.e, pn.n

# these two defs avoid rounding issues with floating point during
# division (especially with large numbers)


def ceildiv(a, b):
    return -(-a // b)


def floordiv(a, b):
    return (a // b)

def pad_message(prefix, nbytes, m):
    prefix_len = len(prefix)
    prefix_num = int(prefix,2)
    return (int.from_bytes(m, "big") + (prefix_num << (nbytes * 8 - prefix_len))).to_bytes(nbytes,"big")


oracle_ctr = 0
def rsa_prefix_padding_oracle_attack(n, e, ct, padding_prefix, oracle):
    print('Generalized Bleichenbacher RSA Padding Oracle Attack')
    print('  for more info see 1998 paper.')
    print()

    # byte length of n
    k = int(ceildiv(math.log(n, 2), 8))
    c = ct

    # lift oracle defition to take integers
    def oracle_int(x):
        global oracle_ctr
        oracle_ctr = oracle_ctr + 1
        if oracle_ctr % 100000 == 0:
            print("[{}K tries] ".format(oracle_ctr // 1000), end='', flush=True)
        return oracle(x)
    
    prefix_nbits = len(padding_prefix)
    prefix_num = int(padding_prefix, 2)
    nbits = n.bit_length()
    B = pow(2, nbits - prefix_nbits)

    # precompute constants
    _lB = prefix_num * B
    _uB = (prefix_num + 1) * B
    padding_pos = nbits - prefix_nbits
    print("[+] Testing the strict bounds, valid bounds should output : ttff")
    print((_lB >> padding_pos) == prefix_num)
    print(((_uB - 1) >> padding_pos) == prefix_num)
    print((_lB-1 >> padding_pos) == prefix_num)
    print(((_uB) >> padding_pos) == prefix_num)

    def multiply(x, y): return (x * pow(y, e, n)) % n

    # should be identity as c is valid cipher text
    c0 = multiply(c, 1)
    assert c0 == c
    i = 1
    M = [(_lB, _uB - 1)]
    s = 1

    # const_s : to enlarge the plaintext if the plaintext is too short
    const_s = None
    # ensure everything is working as expected
    if oracle_int(c0):
        # plaintext is padded correctly
        print('Oracle ok, implicit step 1 passed')
    else:
        # plaintext is not padded and might be too short
        const_s = 2**(nbits - prefix_nbits)
        c0 = multiply(c0, const_s)
        s = 1
        while not oracle_int(multiply(c0, s)):
            s += 1
        c0 = multiply(c0, s)
        assert oracle_int(c0)
        const_s *= s
        print(f"case 1 done {s} times")

    while True:
        if i == 1:
            print('start case 2.a: ', end='', flush=True)
            ss = ceildiv(n, _uB)
            while not oracle_int(multiply(c0, ss)):
                ss = ss + 1
            print('done. found s1 in {} iterations: {}'.format(
                ss - ceildiv(n, _uB), ss))
        else:
            assert i > 1
            if len(M) > 1:
                print('start case 2.b: ', end='', flush=True)
                ss = s + 1
                while not oracle_int(multiply(c0, ss)):
                    ss = ss + 1
                print('done. found s{} in {} iterations: {}'.format(
                    i, ss-s, ss))
            else:
                print('start case 2.c: ', end='', flush=True)
                assert len(M) == 1
                a, b = M[0]
                r = ceildiv(2 * (b * s - _lB), n)
                ctr = 0
                while True:
                    # note: the floor function below needed +1 added
                    # to it, this is not clear from the paper (see
                    # equation 2 in paper where \lt is used instead of
                    # \lte).
                    for ss in range(
                            ceildiv(_lB + r * n, b),
                            floordiv(_uB + r * n, a) + 1):
                        ctr = ctr + 1
                        if oracle_int(multiply(c0, ss)):
                            break
                    else:
                        r = r + 1
                        continue
                    break
                print('done. found s{} in {} iterations: {}'.format(i, ctr, ss))
        # step 3, narrowing solutions
        MM = []
        for a, b in M:
            for r in range(ceildiv(a * ss - _uB + 1, n),
                           floordiv(b * ss - _lB, n) + 1):
                m = (
                    max(a, ceildiv(_lB + r * n, ss)),
                    min(b, floordiv(_uB - 1 + r * n, ss))
                )
                if m not in MM:
                    MM.append(m)
                    print('found interval [{},{}]'.format(m[0], m[1]))
        # step 4, compute solutions
        M = MM
        s = ss
        i = i + 1
        if len(M) == 1 and M[0][0] == M[0][1]:
            print()
            print('Completed!')
            print('used the oracle {} times'.format(oracle_ctr))
            # note, no need to find multiplicative inverse of s0 in n
            # as s0 = 1, so M[0][0] is directly the message.
            if const_s != None:
                message = (M[0][0]*inverse(const_s, n) % n)
            else:
                message = M[0][0]
            m_len = (message.bit_length()-1)//8 + 1
            print(message.to_bytes(m_len, 'big'))
            print('raw decryption in hex format: {}'.format(
                hex(message)))
            return
        
if __name__ == "__main__":
    # note : if the prefix is too long, time cost may be intolerable
    # padded bersion
    # oracle_prfix = bin(11451)[2:].zfill(16)
    # choose_plaintext = pad_message(oracle_prfix , 1024//8 , b"flag{this_is_a_sample_flag_for_testing!}")
    
    # unpadded bersion
    oracle_prfix = bin(1145)[2:].zfill(11)
    choose_plaintext = b"flag{this_is_a_sample_flag_for_testing!}"
    
    print(choose_plaintext)
    ciphertext, oracle, e, n = local_setup(oracle_prfix,choose_plaintext)
    rsa_prefix_padding_oracle_attack(n,e,ciphertext,oracle_prfix,oracle)

b'flag{this_is_a_sample_flag_for_testing!}'
Using local loop back oracle for testing
  keysize: 1024
  e: 65537
  n: 141760948008962981950726708075629548459906389689039681764519975628528854709503436189007969967178587211161291564257703553379437103287464244797515395355532870425957810124201977548234865885963986910656920413275150221121227653986517610451793255933596452619396231397689420136595219835108219874569906950151608190533
  p: 12176114410869395161003693960058391473747968454293810302395135022922884490673426218660827568326329868085620481982956810281474339892695537705439023761363237
  q: 11642544019002939913968256888041800866296898549025133915709381502341719936225215609307930516672240023905244701796127294603152176627318718751018480926402209
  d: 769403073176802915602995102957130024065622515714656069146280045334203787481428387818028516980109304225247896751552026396342001886557990629331159896363321994266084647688924687525889575757190209960197651867700879714913600560037988711579789319748072

start case 2.c: done. found s10 in 2 iterations: 1386273
found interval [100524378055014956464955998764077228464976182013046677189734980003350076306601600613010124787177305381881183685649150241162634046671743619280829076263309428671369771582891301627266563386118400841045824776943878043573292822465972064708176969613987706042245597218232641351921152688362180209880584633075265673161,100524378106991959942999200115155897625040536580850508381883582725402567278616083681405094960366430639480940912031500355401718712457038513294893435638672540711661477688713209778978527816555434441085787965801893336784687397556877481436078105748884128224541359819179895005116729778325406444069854173941401567050]
start case 2.c: done. found s11 in 1 iterations: 2773401
found interval [10052437807174945048218966835751377058632963244450141521626281199209862878135389585629957774225240101362332537253543398187795797515444871115905274313268589483336893661471534060690409123004909292600012570133123316244197337382566560793

start case 2.c: done. found s22 in 2 iterations: 5683602289
found interval [100524378099474796296249664787007435824043544170451827199262390709045746722135374418739236516194162268987065234632906768987820778844560316372897608548606565030206204614496847489129225480650719969581631342447159012058673640296457530818431327041819472000282135534928549911836858021101848018364723619609365411790,100524378099490240372143325371556492222669221363453814584128636284787471889108136744985839019657022495728131248145065898109004429241939366858708952123855727869631602475018018797147072628124231278663425356129995045672671249959793617785592890640775346116337173125437227962224139113534646537350065364547382905229]
start case 2.c: done. found s23 in 3 iterations: 11367207145
found interval [1005243780994837650805945957267872919561163719276515368823166225108421934430942062330436074855706646751360648758213849123072260508465241605991722144962672661701821255196408149840038247747306165919347529859229282535511877858394

start case 2.c: done. found s34 in 2 iterations: 23280042881089
found interval [100524378099489094810607404116506366536848522123023555652519169734219336182584635499201146154006561854961682765038809932103947932969987516783838065940833441474066144762210993626988957901021397159395921468403794246248060557182069899496587221852694020826696857842632765130779628238272051529085969763394384873289,100524378099489098581132464961794514942289140683508934259257955844847607659271779284266451350421691365406773343193706963502582647974458965364394649366256252904702738601799382870568712659352919598482603113501305250174934368613827772824614293703022260576018511719942466851348421281425330370909445830786972576846]
start case 2.c: done. found s35 in 2 iterations: 46560085763889
found interval [100524378099489096403424807730672063978915684923278491796492439403584104409703911411755525412993701390540430928415595382519899655783523370677615913961372560915001985301120198297891679330370243027281012317477748549193756

start case 2.c: done. found s46 in 3 iterations: 95355055648677257
found interval [100524378099489098244044901297688276574297149502926510008034600021931456414373063966877417243150882892253489011719921074020121668856644347587665085787026944583452424822550117063664482553428030795864778456826241083736080732439703502637447682631241538014460502308014312295176208148707006425797640691385999881353,100524378099489098244816816084231195774249613926146363231172448711299951443766968990514515568501157070124280762142194750288570206915366963128460110768406950052172394523516786834277233560648620078069944244426106294713263288589744614100064890584995966287484677281650220149739816047960292272932605952910849068925]
start case 2.c: done. found s47 in 1 iterations: 190710111297355369
found interval [10052437809948909824428806321886383109261282671595269746166441610285852545395906085588068898691514898303098101084140073632383169550230259433076175613719467342760148286494286240914746786884133888970093358271862987

...

start case 2.c: done. found s963 in 1 iterations: 105645194693226377747863454636715027211247423193973369183658662647647822967818992660625109734286664763077299326443390545914027325962436641653206987412393315227735832935780786751978451880375246496068722883698934774477240829825191770856161694902267431999584967948733217786932854754708077015228793
found interval [100524378099489098244696791383846654967553792243460903587618240870574621170612385518592853723003039912387034000229174351678570781562391448486553837962411077338139763595151579561441095970053211336958818845013009540836837478644138099343773867765623297130872186106515050692330930897725522516163273053952947787331,100524378099489098244696791383846654967553792243460903587618240870574621170612385518592853723003039912387034000229174351678570781562391448486553837962411077338139763595151579561441095970053211336958818845013009540836837478644138099343773867765623297130872186106515050692330930897725522516163273054783823108783]
start case 2.c: do

start case 2.c: done. found s971 in 2 iterations: 27045169841465952703453044386999046966079340337657182511016617637797842679761662121120028091977386179347788627569507979753990995446383780263220988777572688698300373231559881408506483681376063102993593058226927302266173652435249093339177393894980462591893751794875703753454810817205267715899053537
found interval [100524378099489098244696791383846654967553792243460903587618240870574621170612385518592853723003039912387034000229174351678570781562391448486553837962411077338139763595151579561441095970053211336958818845013009540836837478644138099343773867765623297130872186106515050692330930897725522516163273054707677908344,100524378099489098244696791383846654967553792243460903587618240870574621170612385518592853723003039912387034000229174351678570781562391448486553837962411077338139763595151579561441095970053211336958818845013009540836837478644138099343773867765623297130872186106515050692330930897725522516163273054710378954052]
start case 2.c: 

start case 2.c: done. found s980 in 3 iterations: 13847126958830567784167958726143512046632622252880477445640508230552495452037971006013454383092421723826067777315588085634043389668548495494769146254117216613529791094558659281155319644864544308732719645812186778760280910046847535789658825674229996847049600918976360321768863138409097070540316085817
found interval [100524378099489098244696791383846654967553792243460903587618240870574621170612385518592853723003039912387034000229174351678570781562391448486553837962411077338139763595151579561441095970053211336958818845013009540836837478644138099343773867765623297130872186106515050692330930897725522516163273054709947113189,100524378099489098244696791383846654967553792243460903587618240870574621170612385518592853723003039912387034000229174351678570781562391448486553837962411077338139763595151579561441095970053211336958818845013009540836837478644138099343773867765623297130872186106515050692330930897725522516163273054709952428801]
start case 2.

start case 2.c: done. found s989 in 2 iterations: 7089729002921250705493994867785478167875902593474804452167940214042877671443441155078888644143319922598946701985581099844630215510296829693321802882108014906127253040414033551951523658170646686071152458655839630725263825943985938324305318745205758385689395670515896484745657926865457700116641836762977
found interval [100524378099489098244696791383846654967553792243460903587618240870574621170612385518592853723003039912387034000229174351678570781562391448486553837962411077338139763595151579561441095970053211336958818845013009540836837478644138099343773867765623297130872186106515050692330930897725522516163273054709952231144,100524378099489098244696791383846654967553792243460903587618240870574621170612385518592853723003039912387034000229174351678570781562391448486553837962411077338139763595151579561441095970053211336958818845013009540836837478644138099343773867765623297130872186106515050692330930897725522516163273054709952243524]
start case 

start case 2.c: done. found s998 in 3 iterations: 3629941249495680361212925372306164821952462127859099879509985389589953367779041871400390985801379800370660711416617523120450670341271976802980763075639303631937153556691985178599180112983371103268430058831789890931335078883320800422044323197545348293472970583304139000189776858555114342459720620423637529
found interval [100524378099489098244696791383846654967553792243460903587618240870574621170612385518592853723003039912387034000229174351678570781562391448486553837962411077338139763595151579561441095970053211336958818845013009540836837478644138099343773867765623297130872186106515050692330930897725522516163273054709952242648,100524378099489098244696791383846654967553792243460903587618240870574621170612385518592853723003039912387034000229174351678570781562391448486553837962411077338139763595151579561441095970053211336958818845013009540836837478644138099343773867765623297130872186106515050692330930897725522516163273054709952242667]
start ca

17